# Ethereum Price Prediciton 

## Modeling and Preprocessing

### 1. Preprocessing

Since our objective is time-series analysis, our primary focus here will be how to best split our test and train data sets. Some considerations here are that we will likely want our test data split to be analogous with the time nearest where we are forecasting from, either by seasonality or simply the most recent time nearest where we are forecasting from.

In [1]:
#import relevant modules
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import statsmodels.api as sm
%matplotlib inline

In [2]:
data = pd.read_csv('data_enhanced.csv')
features = pd.read_csv('engineered_features.csv')

['C:\\springboard_\\Capstone Three',
 'C:\\Springboard_\\DataScienceGuidedCapstone\\Notebooks\\library',
 'C:\\Users\\nicho\\anaconda3\\python38.zip',
 'C:\\Users\\nicho\\anaconda3\\DLLs',
 'C:\\Users\\nicho\\anaconda3\\lib',
 'C:\\Users\\nicho\\anaconda3',
 '',
 'C:\\Users\\nicho\\AppData\\Roaming\\Python\\Python38\\site-packages',
 'C:\\Users\\nicho\\anaconda3\\lib\\site-packages',
 'C:\\Users\\nicho\\anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\nicho\\anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\nicho\\anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\nicho\\anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\nicho\\.ipython']

### 2. Modeling

We will be testing and considering several time-series models, including: exponential smoothing, tbats and Facebook Prophet models. For exponential smoothing, we will likely look to use either the double or triple (also known as Holt_winters Exponential Smoothing) variants since we know our time-series has trend and we will decide between these two based on whether we determine if seasonality is present as well.